In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.2 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GINNet(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_layers, output_dim):
        super(GINNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GINConv(torch.nn.Sequential(torch.nn.Linear(num_features, hidden_dim), torch.nn.ReLU())))
        for _ in range(num_layers - 1):
            self.convs.append(GINConv(torch.nn.Sequential(torch.nn.Linear(hidden_dim, hidden_dim), torch.nn.ReLU())))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
        x = F.relu(x)
        return self.fc(x)


# Define model
model = GINNet(num_features=X_train.shape[1], hidden_dim=128, num_layers=2, output_dim=13).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=10):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []
    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)

            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')
        np.savetxt('/content/drive/MyDrive/PROJECT/train_f1_scores_GIN.txt',train_f1_scores)
        np.savetxt('/content/drive/MyDrive/PROJECT/train_loss_GIN.txt', train_losses)
        np.savetxt('/content/drive/MyDrive/PROJECT/epochs_GIN.txt', epochss)


# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=10)

# Load the best model
model.load_state_dict(torch.load('best_model.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)


print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/f1_score_GIN.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")






Using device: cpu
Epoch [1/500], Loss: 3.0317959785461426, Train Acc: 0.0291, Val Loss: 2.1983256340026855, Val Acc: 0.6787
Epoch [2/500], Loss: 2.664438009262085, Train Acc: 0.6441, Val Loss: 1.9822043180465698, Val Acc: 0.6787
Epoch [3/500], Loss: 2.2680344581604004, Train Acc: 0.6444, Val Loss: 1.8471026420593262, Val Acc: 0.4618
Epoch [4/500], Loss: 2.358961582183838, Train Acc: 0.4313, Val Loss: 1.654290795326233, Val Acc: 0.6807
Epoch [5/500], Loss: 1.9083997011184692, Train Acc: 0.6347, Val Loss: 1.5664993524551392, Val Acc: 0.6787
Epoch [6/500], Loss: 1.798446536064148, Train Acc: 0.6442, Val Loss: 1.5926463603973389, Val Acc: 0.6232
Epoch [7/500], Loss: 1.777637004852295, Train Acc: 0.5896, Val Loss: 1.5116597414016724, Val Acc: 0.6299
Epoch [8/500], Loss: 1.7506234645843506, Train Acc: 0.5848, Val Loss: 1.4020618200302124, Val Acc: 0.6787
Epoch [9/500], Loss: 1.7494885921478271, Train Acc: 0.6440, Val Loss: 1.3518710136413574, Val Acc: 0.6787
Epoch [10/500], Loss: 1.702452659

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GIN + DECISION TREE


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Train a decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth=2)
decision_tree.fit(train_outputs, y_train.cpu().numpy())

# Predict labels using the decision tree
train_pred = decision_tree.predict(train_outputs)
val_pred = decision_tree.predict(val_outputs)
test_pred = decision_tree.predict(test_outputs)

# Evaluate decision tree performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (Decision Tree): {train_acc}')
print(f'Validation Accuracy (Decision Tree): {val_acc}')
print(f'Test Accuracy (Decision Tree): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/f1_scores_GIN_DT.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")


Training Accuracy (Decision Tree): 0.7230782113548819
Validation Accuracy (Decision Tree): 0.7101740294511378
Test Accuracy (Decision Tree): 0.7017426273458445
Precision: 0.6494, Recall: 0.7017, F1-score: 0.6670
F1-score saved to file.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


GIN + DECISION TREE(COMBINED INPUT)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Concatenate the GIN model output with the original feature matrices
X_train_combined = np.concatenate((X_train.cpu().numpy(), train_outputs), axis=1)
X_val_combined = np.concatenate((X_val.cpu().numpy(), val_outputs), axis=1)
X_test_combined = np.concatenate((X_test.cpu().numpy(), test_outputs), axis=1)

# Train a decision tree classifier
decision_tree = DecisionTreeClassifier(max_depth=9)
decision_tree.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the decision tree
train_pred = decision_tree.predict(X_train_combined)
val_pred = decision_tree.predict(X_val_combined)
test_pred = decision_tree.predict(X_test_combined)

# Evaluate decision tree performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (Decision Tree): {train_acc}')
print(f'Validation Accuracy (Decision Tree): {val_acc}')
print(f'Test Accuracy (Decision Tree): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')

with open('/content/drive/MyDrive/PROJECT/f1_scores_GIN_DT_COMB.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")



Training Accuracy (Decision Tree): 0.9151733377993636
Validation Accuracy (Decision Tree): 0.8641231593038822
Test Accuracy (Decision Tree): 0.868632707774799
Precision: 0.8651, Recall: 0.8686, F1-score: 0.8634
F1-score saved to file.


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(train_outputs, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(train_outputs)
val_pred = svm_classifier.predict(val_outputs)
test_pred = svm_classifier.predict(test_outputs)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/f1_scores_GIN_SVM.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")


Training Accuracy (SVM): 0.8725506615307319
Validation Accuracy (SVM): 0.8507362784471219
Test Accuracy (SVM): 0.8558981233243967
Precision: 0.8512, Recall: 0.8559, F1-score: 0.8524
F1-score saved to file.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Load the best GIN model
model.load_state_dict(torch.load('best_model.pt'))

# Get the output of the GIN model for the training, validation, and test sets
model.eval()
with torch.no_grad():
    train_outputs = model(X_train, edge_train).cpu().numpy()
    val_outputs = model(X_val, edge_val).cpu().numpy()
    test_outputs = model(X_test, edge_test).cpu().numpy()

# Combine the output of the GIN model with the original features
X_train_combined = np.concatenate((X_train.cpu().numpy(), train_outputs), axis=1)
X_val_combined = np.concatenate((X_val.cpu().numpy(), val_outputs), axis=1)
X_test_combined = np.concatenate((X_test.cpu().numpy(), test_outputs), axis=1)

# Train an SVM classifier
svm_classifier = SVC(kernel='linear')  # You can specify different kernel functions (e.g., 'linear', 'poly', 'rbf', etc.)
svm_classifier.fit(X_train_combined, y_train.cpu().numpy())

# Predict labels using the SVM classifier
train_pred = svm_classifier.predict(X_train_combined)
val_pred = svm_classifier.predict(X_val_combined)
test_pred = svm_classifier.predict(X_test_combined)

# Evaluate SVM performance
train_acc = accuracy_score(y_train.cpu().numpy(), train_pred)
val_acc = accuracy_score(y_val.cpu().numpy(), val_pred)
test_acc = accuracy_score(y_true.cpu().numpy(), test_pred)

print(f'Training Accuracy (SVM): {train_acc}')
print(f'Validation Accuracy (SVM): {val_acc}')
print(f'Test Accuracy (SVM): {test_acc}')

from sklearn.metrics import precision_recall_fscore_support, roc_auc_score, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), test_pred, average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')

with open('/content/drive/MyDrive/PROJECT/f1_scores_GIN_SVM_COMB.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")


Training Accuracy (SVM): 0.9024451515659019
Validation Accuracy (SVM): 0.8708165997322623
Test Accuracy (SVM): 0.8726541554959786
Precision: 0.8716, Recall: 0.8727, F1-score: 0.8711
F1-score saved to file.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GINNet(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_layers, output_dim):
        super(GINNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GINConv(torch.nn.Sequential(torch.nn.Linear(num_features, hidden_dim), torch.nn.ReLU())))
        for _ in range(num_layers - 1):
            self.convs.append(GINConv(torch.nn.Sequential(torch.nn.Linear(hidden_dim, hidden_dim), torch.nn.ReLU())))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
        x = F.relu(x)
        return self.fc(x)


# Define model
model = GINNet(num_features=X_train.shape[1], hidden_dim=128, num_layers=1, output_dim=13).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=10):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []
    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)

            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model1.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')
        np.savetxt('/content/drive/MyDrive/PROJECT/layer1/train_f1_scores_GIN.txt',train_f1_scores)
        np.savetxt('/content/drive/MyDrive/PROJECT/layer1/train_loss_GIN.txt', train_losses)
        np.savetxt('/content/drive/MyDrive/PROJECT/layer1/epochs_GIN.txt', epochss)


# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=10)

# Load the best model
model.load_state_dict(torch.load('best_model1.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)


print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/layer1/f1_score_GIN.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")






Using device: cpu
Epoch [1/500], Loss: 2.681000232696533, Train Acc: 0.0265, Val Loss: 2.1388442516326904, Val Acc: 0.4665
Epoch [2/500], Loss: 2.1094863414764404, Train Acc: 0.4824, Val Loss: 1.663632869720459, Val Acc: 0.6888
Epoch [3/500], Loss: 1.6862798929214478, Train Acc: 0.6647, Val Loss: 1.2600630521774292, Val Acc: 0.7095
Epoch [4/500], Loss: 1.349033236503601, Train Acc: 0.6763, Val Loss: 1.0377941131591797, Val Acc: 0.7175
Epoch [5/500], Loss: 1.198499321937561, Train Acc: 0.6738, Val Loss: 0.9638200998306274, Val Acc: 0.7142
Epoch [6/500], Loss: 1.1645467281341553, Train Acc: 0.6671, Val Loss: 0.9305998682975769, Val Acc: 0.7175
Epoch [7/500], Loss: 1.1316581964492798, Train Acc: 0.6779, Val Loss: 0.9122809767723083, Val Acc: 0.7329
Epoch [8/500], Loss: 1.0889737606048584, Train Acc: 0.6944, Val Loss: 0.9195653200149536, Val Acc: 0.7323
Epoch [9/500], Loss: 1.0638959407806396, Train Acc: 0.6916, Val Loss: 0.9236361980438232, Val Acc: 0.7262
Epoch [10/500], Loss: 1.04179418

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GINNet(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_layers, output_dim):
        super(GINNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GINConv(torch.nn.Sequential(torch.nn.Linear(num_features, hidden_dim), torch.nn.ReLU())))
        for _ in range(num_layers - 1):
            self.convs.append(GINConv(torch.nn.Sequential(torch.nn.Linear(hidden_dim, hidden_dim), torch.nn.ReLU())))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
        x = F.relu(x)
        return self.fc(x)


# Define model
model = GINNet(num_features=X_train.shape[1], hidden_dim=128, num_layers=3, output_dim=13).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=10):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []
    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)

            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model3.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')
        np.savetxt('/content/drive/MyDrive/PROJECT/layer3/train_f1_scores_GIN.txt',train_f1_scores)
        np.savetxt('/content/drive/MyDrive/PROJECT/layer3/train_loss_GIN.txt', train_losses)
        np.savetxt('/content/drive/MyDrive/PROJECT/layer3/epochs_GIN.txt', epochss)


# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=10)

# Load the best model
model.load_state_dict(torch.load('best_model3.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)


print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/layer3/f1_score_GIN.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")






Using device: cpu
Epoch [1/500], Loss: 2.5974857807159424, Train Acc: 0.0713, Val Loss: 2.2117459774017334, Val Acc: 0.6787
Epoch [2/500], Loss: 4.825202465057373, Train Acc: 0.6441, Val Loss: 2.4012932777404785, Val Acc: 0.1332
Epoch [3/500], Loss: 5.4709367752075195, Train Acc: 0.1383, Val Loss: 2.3385980129241943, Val Acc: 0.6412
Epoch [4/500], Loss: 2.792297124862671, Train Acc: 0.5482, Val Loss: 2.2713801860809326, Val Acc: 0.6787
Epoch [5/500], Loss: 2.5263006687164307, Train Acc: 0.6441, Val Loss: 2.219942808151245, Val Acc: 0.6787
Epoch [6/500], Loss: 2.477268934249878, Train Acc: 0.6441, Val Loss: 2.1745407581329346, Val Acc: 0.6787
Epoch [7/500], Loss: 2.3570504188537598, Train Acc: 0.6441, Val Loss: 2.120152235031128, Val Acc: 0.6787
Epoch [8/500], Loss: 2.238430976867676, Train Acc: 0.6441, Val Loss: 2.0777242183685303, Val Acc: 0.6760
Epoch [9/500], Loss: 2.1314637660980225, Train Acc: 0.6271, Val Loss: 1.9868059158325195, Val Acc: 0.6767
Epoch [10/500], Loss: 2.0329854488

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [5]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GINConv
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score as calculate_f1_score

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load data
edge_index_train = torch.load('/content/drive/MyDrive/PROJECT/edge_index.pt')
edge_index_test = torch.load('/content/drive/MyDrive/PROJECT/edge_index_test.pt')
edge_index_val = torch.load('/content/drive/MyDrive/PROJECT/edge_index_val.pt')

features_file_train = '/content/drive/MyDrive/PROJECT/feature_matrix_train.txt'
X_train = np.loadtxt(features_file_train)
features_file_test = '/content/drive/MyDrive/PROJECT/feature_matrix_test.txt'
X_test = np.loadtxt(features_file_test)
features_file_val = '/content/drive/MyDrive/PROJECT/feature_matrix_val.txt'
X_val = np.loadtxt(features_file_val)

labels_test = pd.read_csv("/content/drive/MyDrive/PROJECT/test_filtered.csv")
labels_train = pd.read_csv("/content/drive/MyDrive/PROJECT/train_filtered.csv")
labels_val = pd.read_csv("/content/drive/MyDrive/PROJECT/val_filtered.csv")

y_train = torch.tensor(labels_train['label'].values, dtype=torch.long).to(device)
y_val = torch.tensor(labels_val['label'].values, dtype=torch.long).to(device)
y_true = torch.tensor(labels_test['label'].values, dtype=torch.long).to(device)

# Preprocess features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_train = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_test_scaled = scaler.transform(X_test)
X_test = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_val_scaled = scaler.transform(X_val)
X_val = torch.tensor(X_val_scaled, dtype=torch.float32).to(device)

class GINNet(torch.nn.Module):
    def __init__(self, num_features, hidden_dim, num_layers, output_dim):
        super(GINNet, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(GINConv(torch.nn.Sequential(torch.nn.Linear(num_features, hidden_dim), torch.nn.ReLU())))
        for _ in range(num_layers - 1):
            self.convs.append(GINConv(torch.nn.Sequential(torch.nn.Linear(hidden_dim, hidden_dim), torch.nn.ReLU())))

        # Output layer
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        for conv in self.convs:
            x = conv(x, edge_index)
        x = F.relu(x)
        return self.fc(x)


# Define model
model = GINNet(num_features=X_train.shape[1], hidden_dim=128, num_layers=1, output_dim=13).to(device)

# Define optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

# Training
def train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=10):
    best_val_loss = float('inf')
    best_val_acc = 0.0
    current_patience = 0
    train_losses = []
    val_losses = []
    train_f1_scores = []  # Initialize list for training F1 scores
    epochss = []

    for epoch in range(epochs):
        epochss.append(epoch)
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train, edge_train)
        loss = criterion(outputs, y_train)
        train_losses.append(loss.cpu().item())
        loss.backward()
        optimizer.step()

        # Compute training accuracy
        _, predicted_train = torch.max(outputs, 1)
        train_acc = torch.sum(predicted_train == y_train).item() / len(y_train)
        # Calculate training F1 score
        train_f1 = calculate_f1_score(y_train.cpu().numpy(), predicted_train.cpu().numpy(), average='weighted')


        # Save training F1 score
        train_f1_scores.append(train_f1)

        # Validation
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_val, edge_val)
            val_loss = criterion(val_outputs, y_val)

            # Compute validation accuracy
            _, predicted_val = torch.max(val_outputs, 1)
            val_acc = torch.sum(predicted_val == y_val).item() / len(y_val)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_model4.pt')
            current_patience = 0
        else:
            current_patience += 1
            if current_patience >= patience:
                print(f'Early stopping at epoch {epoch}')
                break

        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item()}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss.item()}, Val Acc: {val_acc:.4f}')
        np.savetxt('/content/drive/MyDrive/PROJECT/layer4/train_f1_scores_GIN.txt',train_f1_scores)
        np.savetxt('/content/drive/MyDrive/PROJECT/layer4/train_loss_GIN.txt', train_losses)
        np.savetxt('/content/drive/MyDrive/PROJECT/layer4/epochs_GIN.txt', epochss)


# Convert data to appropriate format
edge_train = edge_index_train.to(device)
edge_val = edge_index_val.to(device)
edge_test = edge_index_test.to(device)

# Train the model
train(model, optimizer, criterion, X_train, edge_train, y_train, X_val, edge_val, y_val, epochs=500, patience=10)

# Load the best model
model.load_state_dict(torch.load('best_model4.pt'))

# Testing
model.eval()
with torch.no_grad():
    test_outputs = model(X_test, edge_test)
    test_loss = criterion(test_outputs, y_true)
    _, predicted = torch.max(test_outputs, 1)
    accuracy = torch.sum(predicted == y_true).item() / len(y_true)


print(f'Test Loss: {test_loss.item()}, Test Accuracy: {accuracy}')
from sklearn.metrics import precision_recall_fscore_support, roc_curve, auc

# Calculate precision, recall, F1-score
precision, recall, f1_score, _ = precision_recall_fscore_support(y_true.cpu().numpy(), predicted.cpu().numpy(), average='weighted')

print(f'Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1_score:.4f}')
with open('/content/drive/MyDrive/PROJECT/layer4/f1_score_GIN.txt', 'w') as file:
    file.write(f'{f1_score:.4f}')

print("F1-score saved to file.")






Using device: cpu
Epoch [1/500], Loss: 2.6854968070983887, Train Acc: 0.0608, Val Loss: 2.1256093978881836, Val Acc: 0.6292
Epoch [2/500], Loss: 2.1139743328094482, Train Acc: 0.5862, Val Loss: 1.7023664712905884, Val Acc: 0.6954
Epoch [3/500], Loss: 1.7262715101242065, Train Acc: 0.6594, Val Loss: 1.332602620124817, Val Acc: 0.7209
Epoch [4/500], Loss: 1.4312572479248047, Train Acc: 0.6800, Val Loss: 1.0852720737457275, Val Acc: 0.7088
Epoch [5/500], Loss: 1.2455816268920898, Train Acc: 0.6679, Val Loss: 0.9688936471939087, Val Acc: 0.7062
Epoch [6/500], Loss: 1.1738702058792114, Train Acc: 0.6628, Val Loss: 0.9252703189849854, Val Acc: 0.7162
Epoch [7/500], Loss: 1.1190967559814453, Train Acc: 0.6741, Val Loss: 0.9112683534622192, Val Acc: 0.7336
Epoch [8/500], Loss: 1.082969307899475, Train Acc: 0.6967, Val Loss: 0.916183352470398, Val Acc: 0.7390
Epoch [9/500], Loss: 1.0565087795257568, Train Acc: 0.7035, Val Loss: 0.9237946271896362, Val Acc: 0.7356
Epoch [10/500], Loss: 1.0356868

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
